In [11]:
%load_ext autoreload
%autoreload 2

import os
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from tqdm import tqdm
from src.utils import ask,get_specific_batch
from src.dpo import run_dpo
import json

import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import wandb
mode = 'offline'

wandb.init(project="wandb_test",
           anonymous="allow",
		   mode=mode)

In [13]:
out_dataset_dir="0718iterative_test"
os.system(f"mkdir -p {out_dataset_dir}")

0

In [14]:

# Load model and tokenizer
model_id = "hatakeyama-llm-team/Tanuki-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


In [15]:
pipe=pipeline("text-generation",model=model,tokenizer=tokenizer)

In [16]:
sft_dataset=load_dataset("kanhatakeyama/0717-calm3-22b-random-genre-inst-sft-tsub-part",split="train")

In [17]:
batch_size=32
max_length=1024

In [19]:

#バッチからデータを取得
batch_id=1
for batch_id in range(100):
    ds=get_specific_batch(sft_dataset, batch_size=batch_size, batch_id=batch_id)

    question_list=[record["prompt"] for record in ds]

    #生徒モデルによる出力の生成
    rejected_ans_list=[ask(prompt,pipe,max_length=max_length) for prompt in tqdm(question_list)]


    #rejectedを反映したデータセットの生成
    new_ds=[]
    for record,rejected in zip(ds,rejected_ans_list):
        record["rejected"]=rejected
        record["batch_id"]=batch_id
        if record["rejected"]!=record["chosen"]:
            new_ds.append(record)

    #書き出し
    with open(f"{out_dataset_dir}/batch.json","a") as f:
        for record in new_ds:
            f.write(json.dumps(record,ensure_ascii=False)+"\n")

    dpo_dataset=Dataset.from_list(new_ds)

    #dpo
    run_dpo(model,tokenizer,dpo_dataset,
            run_name="test")

100%|██████████| 32/32 [05:54<00:00, 11.09s/it]


Map: 100%|██████████| 32/32 [00:00<00:00, 317.87 examples/s]
wandb: WARNING URL not available in offline run
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.666100
4,0.651000
